# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sat Sep 18 20:17:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   65C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2020년 Hyundai Data**

In [5]:
lexicon_2020 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2020_hyundai.csv')

lexicon_2020.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,현대차,5380,매일경제,2020010106,SUVs stay popular in S. Korea accounting 44.3%...,http://premium.mk.co.kr/view.php?no=27455,[Pulse로 배우는 영작문-762] 'Pulse로 배우는 영작문'으로 영문 뉴스 ...,2020-01-02,6,121000,121000,118000,118000,556532,-0.020747,-1,0,작문 작문 영문 뉴스 도전 영문 뉴스 원문 바로가기 라이프스타일 변화 글로벌 자동차...


In [6]:
lexicon_2020[lexicon_2020['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2020.dropna(axis=0, inplace=True)
lexicon_2020 = lexicon_2020.reset_index(drop=True)
lexicon_2020.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2020 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2020['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2020['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2020['Tokenization'][x].split())):
                    if lexicon_2020['Tokenization'][x].split()[w] == list(set(lexicon_2020['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2020['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2020['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2020 = pos_lexicon_2020.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2020['date'][x], positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2020['Tokenization'][x].split()))[z])))

***2020-01-02 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2020-01-02 Cosine Similarity between <중요한> & <가장> : 0.5371254086494446
***2020-01-02 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2020-01-02 Cosine Similarity between <추진> & <계획> : 0.5205424427986145
***2020-01-02 Cosine Similarity between <강세> & <급락> : 0.507910966873169
***2020-01-02 Cosine Similarity between <개선> & <향상> : 0.5878475904464722
***2020-01-02 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2020-01-02 Cosine Similarity between <증대> & <향상> : 0.6337747573852539
***2020-01-02 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2020-01-02 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2020-01-02 Cosine Similarity between <증대> & <감소> : 0.6517643928527832
***2020-01-02 Cosine Similarity between <증대> & <급증> : 0.5051040649414062
***2020-01-02 Cosine Similarity between <추진> & <발전> : 0.5099795460700989
***2020-01-02 Cosine Similarity between <추진> & <계획> :

In [11]:
pos_lexicon_2020

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2020-01-02,1,경신,기록,0.522991,2
1,2020-01-02,1,중요한,가장,0.537125,1
2,2020-01-02,1,증대,증가,0.694643,1
3,2020-01-02,1,추진,계획,0.520542,1
4,2020-01-02,2,강세,급락,0.507911,2
...,...,...,...,...,...,...
22181,2021-01-04,4239,본격적인,본격,0.610438,1
22182,2021-01-04,4239,적극적으로,적극,0.663953,1
22183,2021-01-04,4239,중요한,가장,0.537125,1
22184,2021-01-04,4239,추진,계획,0.520542,1


In [12]:
pos_lexicon_2020.to_csv('../../../../Code/Data/Test/Stock-Year/pos_hyundai_2020.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2020['news_word'])))

a

['인더케그',
 '윤자민',
 '코퍼리이션',
 '구현',
 '송출되',
 '라우지츠',
 '오벤터스',
 '몸치장',
 '경제원',
 '스지누스',
 '랙티브',
 '재택명령',
 '어탭티브',
 '쉬장룬',
 '스노체인',
 '라인웍스',
 '거버넌스위',
 '김지정',
 '침체',
 '슬림하',
 '들보',
 '성민석',
 '불경기',
 '우호',
 '젬백스젬백스',
 '이홍락',
 '견조했',
 '개좌터널',
 '리뉴어블',
 '랙하우징',
 '촉진',
 '추진',
 '향상',
 '윤철진',
 '횡보세',
 '커넥티드카로',
 '방법',
 '언택트주',
 '진행',
 '코세스',
 '수펙스홀',
 '샤리오',
 '테레란로',
 '추천',
 '증가',
 '펜데믹',
 '슈웨인스',
 '마슈카',
 '약세',
 '야르베오',
 '딜소싱',
 '휼렛패커드',
 '확보',
 '센티먼트',
 '최고급',
 '언택트',
 '강력',
 '컨설팅펌',
 '페르싱역',
 '본격',
 '발전',
 '컨설트',
 '실패',
 '확대',
 '양수영',
 '인플렉트',
 '엠에스',
 '활황',
 '우세',
 '입관식',
 '텍오스코텍',
 '박승영',
 '활기',
 '연천군',
 '리레이팅',
 '추동훈',
 '액티브후드',
 '완판되자',
 '소형모듈원전',
 '가장',
 '차익',
 '이송량',
 '장밋빛일리',
 '앉음마툰',
 '상쇄출',
 '토픽스지수',
 '소머슨',
 '그린메일',
 '해체설',
 '잇츠한불잇츠한불',
 '콘셉팅하',
 '거버넌',
 '넥쏘특히',
 '확립',
 '응우옌부뚱',
 '지축기지',
 '푸스카스상',
 '웃돈',
 '코리안시리즈',
 '스페인독감',
 '스맨트',
 '요구',
 '김병열',
 '크반트',
 '릴랙션',
 '어드밴티',
 '마련',
 '시너지',
 '극복',
 '해법',
 '배송물량',
 '장단점',
 '감소',
 '킹스컬리지',
 '사대부중',
 '닛케이지수',
 '코람코',
 '딴마음',
 '빠듯하',
 '프론텍',

In [14]:
b = list(set(list(pos_lexicon_2020[pos_lexicon_2020['cosine_similarity']>=0.7]['news_word'])))

b

['최고점', '우세', '밑돈', '추천', '실패', '불황', '추진', '약세', '단점']

In [15]:
c = list(set(list(pos_lexicon_2020[pos_lexicon_2020['cosine_similarity']>=0.65]['news_word'])))

c

['웃돌',
 '송출되',
 '우세',
 '오벤터스',
 '지성규',
 '재택명령',
 '불황',
 '액티브후드',
 '홈컨시어',
 '앉음마툰',
 '소머슨',
 '추진',
 '단점',
 '최고점',
 '적극',
 '밑돈',
 '추천',
 '증가',
 '약세',
 '감소',
 '실패',
 '것이므']